<h3>遷移式學習</h3>
vgg

In [13]:
with open("imagenet_classes.txt", "r") as fp:
    txt = fp.read()
fp.close()

class_names = txt.split("\n")
print(class_names[330:340])

['wood rabbit', 'hare', 'Angora', 'hamster', 'porcupine', 'fox squirrel', 'marmot', 'beaver', 'guinea pig', 'sorrel']


In [6]:
import torch

#下載
vgg16 = torch.hub.load("pytorch/vision:v0.6.0", "vgg16", pretrained=True)
#ANN網路模型執行(原定義啟用)
vgg16.eval()
print(vgg16)

Using cache found in C:\Users\Tony/.cache\torch\hub\pytorch_vision_v0.6.0
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\Tony/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100.0%


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [7]:
from torchvision import transforms

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [10]:
#顯示影像資料
import cv2
from PIL import Image
import matplotlib.pyplot as plt

img = Image.open("rabbit.jpg")
plt.imshow(img)
plt.show

img_tensor = preprocess(img)
print(img_tensor)
print(img_tensor.shape)

tensor([[[-1.2959, -1.2959, -1.2959,  ...,  1.3584,  1.3755,  1.3927],
         [-1.2788, -1.2959, -1.2959,  ...,  1.3755,  1.3927,  1.4098],
         [-1.2788, -1.2788, -1.2788,  ...,  1.3927,  1.4098,  1.4269],
         ...,
         [-0.5424, -0.5596, -0.5938,  ...,  0.9132,  0.8447,  0.8104],
         [-0.4397, -0.4568, -0.4739,  ...,  0.9474,  0.8618,  0.8276],
         [-0.0458, -0.0629, -0.0801,  ...,  1.1358,  1.0502,  1.0331]],

        [[-1.5280, -1.5280, -1.5280,  ...,  0.4153,  0.3978,  0.4153],
         [-1.5105, -1.5280, -1.5280,  ...,  0.4328,  0.4153,  0.4328],
         [-1.5105, -1.5105, -1.5105,  ...,  0.4503,  0.4328,  0.4503],
         ...,
         [-0.2325, -0.2675, -0.3025,  ...,  1.0980,  1.0280,  0.9930],
         [-0.3025, -0.3550, -0.3725,  ...,  1.1331,  1.0455,  1.0105],
         [-0.2150, -0.2500, -0.2850,  ...,  1.3256,  1.2381,  1.2206]],

        [[-1.5430, -1.5430, -1.5430,  ..., -0.3927, -0.3927, -0.3753],
         [-1.5256, -1.5430, -1.5430,  ..., -0

In [11]:
#vgg16所需的Tensor格式(n, 3, 224, 224)
import torch
x = torch.zeros(1, 3, 224, 224)
x[0] = img_tensor
print(x.shape)

torch.Size([1, 3, 224, 224])


In [14]:
#vgg16辨識
import torch
output = vgg16(x)
lst = output[0].tolist()
idx = lst.index(max(lst))
print("class index =", idx)
print("class name =", class_names[idx])

class index = 332
class name = Angora


<h3>訓練辨識 東方龍 與 西方龍</h3>

In [17]:
from PIL import Image
from torchvision import transforms
import glob

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

import numpy as np

images_nd = np.zeros([20, 3, 224, 224], dtype=np.float32)

i=0

files = glob.glob("trainingfile/0/*")
for f in files:
    img = Image.open(f)
    tsr = preprocess(img)
    images_nd[i] = tsr.numpy()
    i = i+1

files = glob.glob("trainingfile/1/*")
for f in files:
    img = Image.open(f)
    tsr = preprocess(img)
    images_nd[i] = tsr.numpy()
    i = i+1
    
input_tensor = torch.from_numpy(images_nd)
print(input_tensor.shape)

torch.Size([20, 3, 224, 224])


In [18]:
#定義label
import torch
import numpy as np

labels_nd = np.zeros([20,2], dtype=np.float32)

for i in range(20):
    if i<10:
        labels_nd[i,0], labels_nd[i,1] = 1, 0
    else:
        labels_nd[i,0], labels_nd[i,1] = 0, 1

output_tensor = torch.from_numpy(labels_nd)
print(output_tensor)

tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.]])


In [19]:
#載入vgg16 model
import torch
import torch.nn as nn

vgg16 = torch.hub.load("pytorch/vision:v0.6.0", "vgg16", pretrained=True)
vgg16.eval()

print(vgg16)
print()

Using cache found in C:\Users\Tony/.cache\torch\hub\pytorch_vision_v0.6.0


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [20]:
#調整model符合應用
#將1000種output改成2種output
vgg16.classifier[-1]=nn.Linear(in_features=4096, out_features=2)
print(vgg16)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [25]:
#move to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

vgg16 = vgg16.to(device)

#訓練樣本Tensor
x = input_tensor
y = output_tensor

cpu


In [26]:
print(x.shape)
print(y.shape)

torch.Size([20, 3, 224, 224])
torch.Size([20, 2])


In [28]:
#開始訓練
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

N = len(x)
print("樣本數：", N)

EPOCH = 100 #訓練回合數

criterion = nn.MSELoss()
optimizer = torch.optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)

for epoch in range(EPOCH):
    #洗牌
    permute = torch.randperm(x.size()[0])
    xi = x[permute]
    yi = y[permute]
    
    optimizer.zero_grad()
    output = vgg16(xi)
    loss = criterion(output, yi)
    print("epoch=%8d, loss=%20.12f" %(epoch, loss))
    loss.backward()
    optimizer.step()

樣本數： 20
epoch=       0, loss=      0.050531316549
epoch=       1, loss=      0.029721623287
epoch=       2, loss=      0.012923799455
epoch=       3, loss=      0.013079400174
epoch=       4, loss=      0.019021708518
epoch=       5, loss=      0.017089482397
epoch=       6, loss=      0.009991925210
epoch=       7, loss=      0.007701641414
epoch=       8, loss=      0.011605360545
epoch=       9, loss=      0.014630896039
epoch=      10, loss=      0.011549333110
epoch=      11, loss=      0.005005658604
epoch=      12, loss=      0.001742303721
epoch=      13, loss=      0.004199404269
epoch=      14, loss=      0.007847016677
epoch=      15, loss=      0.007749748882
epoch=      16, loss=      0.004848761484
epoch=      17, loss=      0.003119782312
epoch=      18, loss=      0.003580294549
epoch=      19, loss=      0.004101675469
epoch=      20, loss=      0.003110264894
epoch=      21, loss=      0.001416889718
epoch=      22, loss=      0.000742020202
epoch=      23, loss=     

In [29]:
#儲存訓練結果
torch.save(vgg16.state_dict(), "vgg16.model")

In [31]:
#載入訓練過的vgg16 model
from PIL import Image
from torchvision import transforms
import torch
import glob

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

files = glob.glob("trainingfile/u/*")

n = len(files)
input_tensor=torch.Tensor(n, 3, 224, 224)

i = 0
for f in files:
    img = Image.open(f)
    tsr = preprocess(img)
    input_tensor[i] = tsr
    i = i+1
    
output = vgg16(input_tensor)
probs = torch.nn.functional.softmax(output, dim=1)

vlst = output.detach().tolist()
plst = probs.detach().tolist()

for i in range(n):
    v0 = vlst[i][0]
    v1 = vlst[i][1]
    p0 = plst[i][0]
    p1 = plst[i][1]
    t = "p0=%.6f ( v0=%.6f), p1=%.6f (v1=%.6f), fileName=%s" %(p0, v0, p1, v1, files[i])
    if(v0>=v1):
        print("chinese dragon!")
    else:
        print("dragon!")
    print(t)
    

chinese dragon!
p0=0.527385 ( v0=0.457598), p1=0.472615 (v1=0.347949), fileName=jupyter/trainingfile/u\0.jpg
dragon!
p0=0.430444 ( v0=0.403845), p1=0.569556 (v1=0.683883), fileName=jupyter/trainingfile/u\1.jpg
